Тренировка модели RandomForestClassifier указанной в примерах библиотеки на трафике,
полученном в пункте 4. В качестве данных для тестирования использовать трафик из
пунктов 1-3. Данный модуль должен быть реализован отдельно

In [2]:
from nfstream import NFStreamer
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import pickle

In [24]:
nfs = NFStreamer(source='data/4/final.pcapng').to_pandas()
#display(nfs)
X_train = nfs[["bidirectional_bytes", 
               "src_port", "bidirectional_duration_ms"]]
X_train = preprocessing.normalize(X_train, axis=0)

Y_train = nfs['application_name'].apply(lambda x: 1 if ('WireGuard' or 'IPsec.Azure' or 'OpenVPN.Azure') in x else 0)
model = RandomForestClassifier(criterion='gini')
model.fit(X_train, Y_train)

with open("model.pkl", "wb") as file:
    pickle.dump(model, file)

In [22]:
def print_result(pcap_file:str, name:str):
    print(name)
    nfs = NFStreamer(source=pcap_file).to_pandas()
    X_test = nfs[["bidirectional_bytes", 
               "src_port", "bidirectional_duration_ms"]]
    X_test = preprocessing.normalize(X_train, axis=0)
    Y_test = model.predict(X_test)

    if 1 in Y_test:
        print('     VPN used')
    else:
        print('     No VPN used')

In [25]:
print_result('data/1/wireguard.pcapng', '1_wireguard')
print_result('data/1/ipsec.pcapng', '1_ipsec')
print_result('data/1/openvpn.pcapng', '1_openvpn')

print_result('data/2/wireguard.pcapng', '2_wireguard')
print_result('data/2/ipsec.pcapng', '2_ipsec')
print_result('data/2/openvpn.pcapng', '2_openvpn')

print_result('data/3/novpn.pcapng', '3_novpn')

1_wireguard
     VPN used
1_ipsec
     VPN used
1_openvpn
     VPN used
2_wireguard
     VPN used
2_ipsec
     VPN used
2_openvpn
     VPN used
3_novpn
     VPN used
